<div style="background: #000;
            color: #FFF;
            margin: 0px;
                padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 10 - Class 2 - 11/17</h1>
</div>

## Objectives for this week:
* Creating Models and Views
* Migration of a Database
* Creating Login Functionality and Forms
* Amazon s3

## Today's Objective:
* More Flask Features (message flashing, blueprints)
* Creating DB and tables
* Select, Update, Delete, Joins
* SQLAlchemy
* Flask-SQLAlchemy
* Creating models in flask.


<div style="background: #000;
            color: #FFF;
            margin: 0px;
                padding: 10px 0px 20px 0px;
            text-align: center; 
                ">
    <h1>Week 9 Homework</h1>
</div>

# More Flask Features

### Message Flashing

Message flashing is a feature of the `flask` library that lets you attach a message to the next response using `flash`. Your template must `get_flashed_messages()` in order to access the messages. This allows you to do things like have a welcome banner upon sign in.

In [ ]:
from flask import Flask, flash, redirect, render_template, \
     request, url_for

layout_html = \
"""
<!doctype html>
<title>My Application</title>
{% with messages = get_flashed_messages() %}
  {% if messages %}
    <ul class=flashes>
    {% for message in messages %}
      <li>{{ message }}</li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
{% block body %}{% endblock %}
"""

index_html = \
"""
{% extends "layout.html" %}
{% block body %}
  <h1>Overview</h1>
  <p>Do you want to <a href="{{ url_for('login') }}">log in?</a>
{% endblock %}
"""

login_html = \
"""
{% extends "layout.html" %}
{% block body %}
  <h1>Login</h1>
  {% if error %}
    <p class=error><strong>Error:</strong> {{ error }}
  {% endif %}
  <form method=post>
    <dl>
      <dt>Username:
      <dd><input type=text name=username value="{{
          request.form.username }}">
      <dt>Password:
      <dd><input type=password name=password>
    </dl>
    <p><input type=submit value=Login>
  </form>
{% endblock %}
"""

app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or \
                request.form['password'] != 'secret':
            error = 'Invalid credentials'
        else:
            flash('You were successfully logged in')
            return redirect(url_for('index'))
    return render_template('login.html', error=error)

# Blueprints

So far, we've been using the same `app.py` file to hold all of our routes/views. This is fine for a small application but as our project gets larger, we might want to modularize your application into parts (e.g. admin, user, etc) you can separate them out into blueprints. Which you can consider submodules of the application.

https://flask.palletsprojects.com/en/1.1.x/blueprints/

Structure for our blueprint application:
```
blueprint_application/
templates/
admin/
    templates/
    admin.py
app.py
```

# Creating DB and tables

Yesterday, we seen some examples of SQL but we haven't seen how to do any of that with python.

# Reference guide to SQL
https://www.w3schools.com/sql/default.asp

### SQLite3 in python

We'll be using the`sqlite3` from the python standard library

Some concepts:
* connection - our connection to the database
* cursor - object that can be used to execute SQL statements or fetch data from queries


In [ ]:
import sqlite3 

# this creates a connection to sqlite3 db in memory
conn = sqlite3.connect(':memory:')
conn.close()

Usually we want to store our db in a file

In [ ]:
import sqlite3 

# this creates a connection to sqlite3 
# db stored in `database.db`
conn = sqlite3.connect('database.db')
conn.close()

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
# this creates a cursor
cursor = conn.cursor()
conn.close()

This deletes a table if it exists and then creates a new table of the same name (students).

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")

cursor = conn.cursor()

cursor.execute("DROP table IF EXISTS students;")

cursor.execute("""
CREATE TABLE students (
 name varchar(255), 
 class varchar(255), 
 grade int
);
""")

print("Table created successfully")

conn.close()

# Insert, Select, Update, Delete

### Insert

Now that we have our table, we can insert rows into it.

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("""
INSERT INTO students (name, class, grade)
VALUES (
"student1", 
"data science", 
100);
""")
# we need to commit these changes
conn.commit()
print("Row created successfully")
conn.close()

In [ ]:
import sqlite3

students = [
    ("student2", "datascience", 93),
    ("student3", "datascience", 100),
    ("student4", "datascience", 98),
    ("student5", "datascience", 80),
    ("student6", "progressiveweb", 90),
    ("student7", "progressiveweb", 100),
    ("student8", "progressiveweb", 80),
    ("student9", "progressiveweb", 98),
    ("student10", "progressiveweb", 93),
]
conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
for student in students:
    print(student)
    cursor.execute(f"INSERT INTO students (name, class, grade) VALUES ('{student[0]}', '{student[1]}', '{student[2]}');")
# we need to commit these changes
conn.commit()
print("Rows created successfully")
conn.close()

### Select

Now that we've got a row in our table, we can select students from the table to get the data.

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("SELECT * FROM students;")
row = cursor.fetchall()
print(row)
print("Operations completed successfully")
conn.close()

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("SELECT * FROM students WHERE grade > 95 ;")
row = cursor.fetchall()
print(row)
print("Operations completed successfully")
conn.close()

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("SELECT * FROM students WHERE name = 'student1' ;")
row = cursor.fetchall()
print(row)
print("Operations completed successfully")
conn.close()

### Update

We can update values in a row using the following:

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("SELECT * FROM students WHERE name = 'student5' ;")
row = cursor.fetchall()
print(row)
print("updating student's grade")
cursor.execute("UPDATE students SET grade = 90 WHERE name = 'student5' ;")
print("printing updated row")
cursor.execute("SELECT * FROM students WHERE name = 'student5' ;")
row = cursor.fetchall()
print(row)
conn.commit()
print("Operations completed successfully")
conn.close()

In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
print("Opened database successfully")
cursor = conn.cursor()
cursor.execute("SELECT * FROM students WHERE name = 'student5' ;")
row = cursor.fetchall()
print(row)
cursor.execute("DELETE FROM students WHERE name = 'student5';")
cursor.execute("SELECT * FROM students WHERE name = 'student5' ;")
row = cursor.fetchall()
print(row)
conn.commit()
print("Operations completed successfully")
conn.close()

# SQLAlchemy

`SQLAlchemy` is a python library that is a wrapper around SQL databases. This is known as an object relational mapping. Essentially, `SQLAlchemy` is python code we can use to complete the same things we did above but without using SQL. This allows us to avoid having to mix python and SQL and just use one.

In [ ]:
!python3 -m pip install sqlalchemy

### What we did above but in SQLAlchemy...

In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
 
Base = declarative_base()
 
class Student(Base):
    __tablename__ = 'student'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
    course = Column(String(255), nullable=False)
    grade = Column(Integer, nullable=False)
    
    def __repr__(self):
        return f"{self.id} {self.name} {self.course} {self.grade}"

# Create an engine that stores data in the .db file
engine = create_engine('sqlite:///sqlalchemy_database.db')
 
# Create all tables in the engine. 
#This is equivalent to "Create Table"
Base.metadata.create_all(engine)
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()
 
# Insert a Student in the student table
new_student = Student(name='student1', course="datascience", grade=90)
session.add(new_student)
session.commit()

In [ ]:
students = [
    ("student2", "datascience", 93),
    ("student3", "datascience", 100),
    ("student4", "datascience", 98),
    ("student5", "datascience", 80),
    ("student6", "progressiveweb", 90),
    ("student7", "progressiveweb", 100),
    ("student8", "progressiveweb", 80),
    ("student9", "progressiveweb", 98),
    ("student10", "progressiveweb", 93),
]

for student in students:
    session.add(
        Student(
            name=student[0], 
            course=student[1],
            grade = student[2]
    ))
    session.commit()

In [ ]:
qry = session.query(Student).order_by('grade')
qry.all()

# Flask-SQLAlchemy

Flask SQLAlchemy is a flask extension that binds `flask` and `SQLAlchemy`. This makes what we were doing above even easier for us.

In [ ]:
!python3 -m pip install flask-sqlalchemy

# Creating models in flask.

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///flask_database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


class Student(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(255), nullable=False)
    course = db.Column(db.String(120), nullable=False)
    grade = db.Column(db.Integer, nullable=False)
    
    def __repr__(self):
        return '<Student %r>' % self.name

db.create_all()

new_student = Student(name='student1', course="datascience", grade=90)
db.session.add(new_student)
db.session.commit()

In [ ]:
Student.query.all()

In [ ]:
students = [
    ("student2", "datascience", 93),
    ("student3", "datascience", 100),
    ("student4", "datascience", 98),
    ("student5", "datascience", 80),
    ("student6", "progressiveweb", 90),
    ("student7", "progressiveweb", 100),
    ("student8", "progressiveweb", 80),
    ("student9", "progressiveweb", 98),
    ("student10", "progressiveweb", 93),
]

for student in students:
    s = Student(name=student[0], course=student[1], grade=student[2])
    db.session.add(s)
    db.session.commit()

In [ ]:
Student.query.all()

In [ ]:
db.close_all_sessions()